# SynFerm Exploratory Data Analysis
#### Targets:
- See some of the basic trends in the data (which BBs react well? which not at all?)
- perform basic statistics (How many successful reactions?)

#### Sections:
1. [Load and filter data](#1)
2. [Investigate "invalid" reactions](#2)
3. [General Statistics](#3)
4. [Building Block Statistics](#4)
5. [Trends within building block classes](#5)
6. [Scaling the LC-MS responses](#6)
7. [Outcome for the main product](#7)

In [ ]:
import pathlib
import re
import sys
from typing import Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import rc
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from rdkit.Chem.AllChem import Compute2DCoords
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score, accuracy_score, balanced_accuracy_score, confusion_matrix, recall_score
%matplotlib inline

sys.path.append(str(pathlib.Path().resolve().parents[1]))
from src.util.db_utils import SynFermDatabaseConnection
from src.util.color_palette import BodeColorPalette
from src.util.pandas_utils import alphanumeric_index_sort

In [ ]:
# settings
bode_palette = BodeColorPalette()
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['figure.dpi'] = 100  # this makes the figures bigger in jupyter nb

sns.set_theme(context="paper",
              style="white",
              font_scale=0.7,
              rc={"savefig.transparent": True,
                  "axes.grid": False,
                  "axes.spines.bottom": True,
                  "axes.spines.left": False,
                  "axes.spines.right": False,
                  "axes.spines.top": False,
                  "axes.labelweight": "bold",
                  },
             )
SAVE_DIR = pathlib.Path("/Users/julian/Desktop/SF_exploratory_data_analysis/")
SAVE_DIR.mkdir(exist_ok=True, parents=True)

### Load and filter data <a id=1></a>

In [ ]:
db = SynFermDatabaseConnection()  # we will use this for various simple queries

In [ ]:
df_full = db.get_experiments_table_as_df()
print(f'Number of reactions (in total): {len(df_full)}')

### Select experiments for analysis

In [ ]:
# show all available experiments with reaction counts
df_full["exp_nr"].value_counts().sort_index()

In [ ]:
# select experiment numbers for further analysis
start_exp_nr = 4  # kick out invalid experiments (SOP changed after exp3)
end_exp_nr = 29  # (inclusive)

df_full = df_full.loc[df_full['exp_nr'].between(start_exp_nr, end_exp_nr)]  
print(f'Number of reactions since SOP change and without test reactions: {len(df_full)}')

### Investigate "invalid" reactions <a id=2></a>

In [ ]:
# expand the "valid" column to work with it more easily
# add columns for error and warning
df_full["has_error"] = df_full["valid"].notna() & df_full["valid"].str.contains("ERROR")
df_full["has_warning"] = df_full["valid"].notna() & df_full["valid"].str.contains("WARNING")
# add columns for error categories
df_full["has_error_product_peaks"] = df_full["valid"].notna() & df_full["valid"].str.contains("ERROR: multiple peaks for product A")
df_full["has_error_internal_standard"] = df_full["valid"].notna() & (
        df_full["valid"].str.contains("ERROR: IS response <50% of plate median")
        | df_full["valid"].str.contains("ERROR: multiple peaks for IS")
        | df_full["valid"].str.contains("ERROR: IS response >200% of plate median")
)
df_full["has_error_transfer"] = df_full["valid"].notna() & df_full["valid"].str.contains("transfer error")
df_full["has_error_low_volume"] = df_full["valid"].notna() & df_full["valid"].str.contains("ERROR: Dilution survey low volume")
df_full["has_error_other"] = df_full["valid"].notna() & (df_full["valid"].str.contains("ERROR: No monomer") | df_full["valid"].str.contains("ERROR: Monomer solution had lower concentration than required") )
df_full["has_warning_product_peaks"] = df_full["valid"].notna() & df_full["valid"].str.contains("WARNING: multiple peaks for product A")
df_full["has_warning_quality_control"] = df_full["valid"].notna() & df_full["valid"].str.contains("WARNING") & df_full["valid"].str.contains("QC")
df_full

In [ ]:
df_full[[col for col in df_full.columns if col.startswith("has_")]].sum()

In [ ]:
# let's split this into the individual errors/warnings
expanded_warnings = df_full.loc[:,'valid'].str.split(";", expand=True)
expanded_warnings.columns = [f"problem_{i}" for i in expanded_warnings.columns]
df_full = df_full.join(expanded_warnings, how="left")
# sanitize
df_full.replace(' ', np.nan, inplace=True)  # replace whitespace strings with NaN (artifact from splitting)
df_full

In [ ]:
# obtain IS areas from the database
res = db.con.execute("SELECT exp_nr, plate_nr, lcms_areas FROM experiments INNER JOIN lcms l on experiments.id = l.synthesis_id;").fetchall()
df_lcms_areas = pd.DataFrame(res, columns=["exp_nr", "plate_nr", "lcms_areas"])
df_lcms_areas["IS_area"] = df_lcms_areas["lcms_areas"].apply(lambda x: float(x.strip("[]").split(", ")[-1]))
df_lcms_areas

In [ ]:
grouped = df_lcms_areas.groupby(["exp_nr", "plate_nr"])
df_lcms_is_mean = grouped.mean(numeric_only=True)
df_lcms_is_std = grouped.std(numeric_only=True)
df_lcms_is = df_lcms_is_mean.join(df_lcms_is_std, lsuffix="_mean", rsuffix="_std")
df_lcms_is = df_lcms_is.reset_index()
df_lcms_is["plate"] = df_lcms_is["exp_nr"] * 7 + df_lcms_is["plate_nr"]
df_lcms_is = df_lcms_is.loc[df_lcms_is["exp_nr"].between(4, 30)]

In [ ]:
plt.figure(figsize=(7.5, 5))
plt.bar(
    x=df_lcms_is["plate"],
    height=df_lcms_is["IS_area_mean"],
    #yerr=df_lcms_is["IS_area_std"],
    width=1,

    )
plt.savefig(SAVE_DIR / "IS_areas.png", dpi=300)

In [ ]:
# plot the standard deviations
plt.figure(figsize=(7.5, 5))
plt.bar(
    x=df_lcms_is["plate"],
    height=df_lcms_is["IS_area_std"],
    #yerr=df_lcms_is["IS_area_std"],
    width=1,

    )
plt.savefig(SAVE_DIR / "IS_areas_std.png", dpi=300)

### Conclusion from IS response statistics
There is no clear trend in the IS response. The standard deviation is generally quite high.
I would have expected to see the SD increase with plate number within an experiment block, because that correlates with the age of the plate at measurement time, but we do not observe a trend like this.

In [ ]:
# how many reactions have errors?
df_full.has_error.sum()

In [ ]:
# how many reactions have warnings?
df_full.has_warning.sum()

In [ ]:
# what are the most common problems?
df_full[[col for col in df_full.columns if col.startswith("problem_")]].stack().value_counts()

In [ ]:
# Generate a histogram showing the distribution of errors across experiments
plt.figure(figsize=(7.5, 5))
sns.barplot(x=df_full.loc[df_full.has_error, "exp_nr"].value_counts().sort_index().index,
            y=df_full.loc[df_full.has_error, "exp_nr"].value_counts().sort_index().values,
            color=bode_palette.blue,
            )
plt.xlabel('Experiment number')
plt.ylabel('Number of errors')
plt.axvline(x=8.5, color="black", linestyle='--', label='Changed to OT2 source preparation before exp13')
plt.axvline(x=12.5, color="black", linestyle='--', label='Changed to full automation before exp17')
plt.axvline(x=16.5, color="black", linestyle='--', label='Changed to Eppendorf heat seal before exp21')
plt.axvline(x=20.5, color="black", linestyle='--', label='Changed to Agilent heat seal before exp21')
plt.savefig(SAVE_DIR / "number_of_errors_per_experiment.png", dpi=300, bbox_inches='tight')

In [ ]:
# Generate a histogram showing the distribution of errors across experiments
# Function to remove the value in parentheses
def remove_parentheses_value(s):
    if isinstance(s, str):
        return re.sub(r'\s\(\d+\)', '', s)
    return s


# Keep only the relevant columns
relevant_columns = ['exp_nr'] + [col for col in df_full.columns if 'problem_' in col]
df_relevant = df_full[relevant_columns].copy()

for col in relevant_columns:
    if col != 'exp_nr':
        df_relevant[col] = df_relevant[col].apply(remove_parentheses_value)
        df_relevant[col] = df_relevant[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

# right now we are only interested in errors
df_relevant_errors = df_relevant.replace(re.compile('.*WARNING.*'), pd.NA)

# Melt the DataFrame
df_melted = pd.melt(df_relevant_errors, id_vars=['exp_nr'], var_name='columns', value_name='value')

# Filter out None values
df_filtered = df_melted[df_melted['value'].notnull()]

# Create the histogram
plt.figure(figsize=(7.5, 5))
ax = sns.histplot(data=df_filtered, x='exp_nr', hue='value', multiple='stack', discrete=True, shrink=.8)
plt.axvline(x=12.5, color=bode_palette.orange, linestyle='--', label='Changed to OT2 source preparation before exp13')
plt.axvline(x=16.5, color=bode_palette.orange, linestyle='--', label='Changed to full automation before exp17')
plt.axvline(x=20.5, color=bode_palette.orange, linestyle='--', label='Changed to Eppendorf heat seal before exp21')
plt.axvline(x=24.5, color=bode_palette.orange, linestyle='--', label='Changed to Agilent heat seal before exp25')
plt.xlim(3.5, 29.5)
plt.xlabel('Experiment number')
plt.ylabel('Number of errors')
# Move the legend outside the plot to the right
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.savefig(SAVE_DIR / "type_of_errors_per_experiment.png", dpi=300, bbox_inches='tight')

In [ ]:
# Generate a histogram showing the distribution of errors across experiments, but use the error bins instead of the individual errors

# Keep only the relevant columns
relevant_columns = ['exp_nr'] + [col for col in df_full.columns if 'has_error_' in col]
df_relevant = df_full[relevant_columns].copy()

# Reshape the DataFrame
df_grouped = df_relevant.groupby('exp_nr').sum().reset_index().melt(id_vars=["exp_nr"], var_name="columns", value_name='value')
# Filter out None values
df_grouped = df_grouped[df_grouped['value'].notnull()]
# Plot the histogram
plt.figure(figsize=(9, 6))
ax = sns.barplot(data=df_grouped, x="exp_nr", y="value", hue='columns', dodge=True, width=.95)
plt.axvline(x=8.5, color="black", linestyle='--', label='Changed to OT2 source preparation before exp13')
plt.axvline(x=12.5, color="black", linestyle='--', label='Changed to full automation before exp17')
plt.axvline(x=16.5, color="black", linestyle='--', label='Changed to Eppendorf heat seal before exp21')
plt.axvline(x=20.5, color="black", linestyle='--', label='Changed to Agilent heat seal before exp25')
plt.xlabel('Experiment number')
plt.ylabel('Number of errors')
handles, labels = ax.get_legend_handles_labels()
ax.legend(title = "Error type", loc='upper left', bbox_to_anchor=(1, 1), handles=handles[4:], labels=[l.removeprefix("has_error_").replace("_", " ").strip().capitalize() for l in labels[4:]])
plt.savefig(SAVE_DIR / "type_of_errors_per_experiment_binned.png", dpi=300, bbox_inches='tight')


In [ ]:
# next, we look at the distribution of errors by plate number within an experiment
# Keep only the relevant columns
relevant_columns = ['plate_nr'] + [col for col in df_full.columns if 'has_error_' in col]
df_relevant = df_full[relevant_columns].copy()
df_grouped = df_relevant.groupby('plate_nr').sum().reset_index().melt(id_vars=["plate_nr"], var_name="columns", value_name='value')
# Filter out None values
df_grouped = df_grouped[df_grouped['value'].notnull()]
# Plot the histogram
plt.figure(figsize=(5, 3.75))
ax = sns.barplot(data=df_grouped, x="plate_nr", y="value", hue='columns', dodge=True, width=.5)
plt.xlabel('Plate number')
plt.ylabel('Number of errors')
handles, labels = ax.get_legend_handles_labels()
ax.legend(title = "Error type", loc='upper left', bbox_to_anchor=(1, 1), handles=handles, labels=[l.removeprefix("has_error_").replace("_", " ").strip().capitalize() for l in labels])
plt.savefig(SAVE_DIR / "type_of_errors_per_plate_binned.png", dpi=300, bbox_inches='tight')

Let's have a look at the worst offenders. there might be a structural reason why they give multiple peaks in LCMS

In [ ]:
x = df_full.loc[df_full['valid'].notna() & df_full['valid'].str.contains('multiple peaks for product A'), ['initiator']].value_counts()
x

In [ ]:
# let's look at these molecules
mols = []
for bb in x.head(5).index.to_list():
    mols.append(db.get_mol(short=bb[0]))
from rdkit.Chem import Draw
Draw.MolsToGridImage(mols, subImgSize=(300,300), legends=[bb[0] for bb in x.index.to_list()])

In [ ]:
x = df_full.loc[df_full['valid'].notna() & df_full['valid'].str.contains('multiple peaks for product A'), ['monomer']].value_counts()
x

In [ ]:
# let's look at these molecules
mols = []
for bb in x.head(10).index.to_list():
    mols.append(db.get_mol(short=bb[0]))
from rdkit.Chem import Draw
Draw.MolsToGridImage(mols, subImgSize=(300,300), legends=[bb[0] for bb in x.index.to_list()])

In [ ]:
x = df_full.loc[df_full['valid'].notna() & df_full['valid'].str.contains('multiple peaks for product A'), ['terminator']].value_counts()
x

In [ ]:
# let's look at these molecules
mols = []
for bb in x.head(5).index.to_list():
    mols.append(db.get_mol(short=bb[0]))
from rdkit.Chem import Draw
Draw.MolsToGridImage(mols, subImgSize=(300,300), legends=[bb[0] for bb in x.index.to_list()])

#### Remove invalid reactions
We move all entries containing "ERROR". Warnings can stay.

In [ ]:
df = df_full.loc[~df_full.has_error]
print(f'Number of reactions after removing invalid entries: {len(df)}')

In [ ]:
df = df.loc[~df['product_A_lcms_ratio'].isna()]  # ensure all entries have a measured lcms ratio
print(f'Number of reactions after removing NaN entries: {len(df)}')

Now we have a cleaned dataset (which may still contain a few duplicates). Let's try some basic statistics

How many reactions are successful (i.e. normalized lmcs ratio for A > threshold ?)

In [ ]:
# add min-max normalized LC-MS responses (each column is normalized separately)
lcms_ratio_columns = [f'product_{s}_lcms_ratio' for s in "ABCDEFGH"]

df[[f'{s}_normalized' for s in "ABCDEFGH"]] = (
      df[lcms_ratio_columns] - df[lcms_ratio_columns].min()) / (df[lcms_ratio_columns].max() - df[lcms_ratio_columns].min()
)


In [ ]:
# add information about reactant classes to df
def get_reaction_type_for_series(ser):
    new = []
    for i in ser:
        new.append(db.get_building_block_class(i))
    return pd.Series(data=new)
        
df[['initiator_type', 'monomer_type', 'terminator_type']] = df[['initiator', 'monomer', 'terminator']].apply(get_reaction_type_for_series, axis=1)
df

### General Statistics <a id=3></a>

Let's look at how well reactions have worked across the (cleaned) data set

In [ ]:
df[[col for col in df.columns if 'lcms' in col]]

In [ ]:
for product_type in "ABCDEFGH":
    print(f'Number of reactions with any LC-MS signal for {product_type}: {len(df.loc[df[f"{product_type}_normalized"] > 0])} ({len(df.loc[df[f"{product_type}_normalized"] > 0]) / len(df):.1%})')
    for threshold in [1e-10, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]:
        t_and_f = (df[f"{product_type}_normalized"] > threshold).value_counts()
        print(f'This ratio of reactions is above the {threshold:.1} threshold: {t_and_f[True] / (t_and_f[True]+ t_and_f[False]):.1%}')

For a graphical assessment, we plot the histograms for all normalized product yields

In [ ]:
nbins = 100
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(7.5, 3.75))
sns.histplot(df['A_normalized'], bins=nbins, ax=axs[0][0])
sns.histplot(df['B_normalized'], bins=nbins, ax=axs[0][1])
sns.histplot(df['C_normalized'], bins=nbins, ax=axs[0][2])
sns.histplot(df['D_normalized'], bins=nbins, ax=axs[0][3])
sns.histplot(df['E_normalized'], bins=nbins, ax=axs[1][0])
sns.histplot(df['F_normalized'], bins=nbins, ax=axs[1][1])
sns.histplot(df['G_normalized'], bins=nbins, ax=axs[1][2])
sns.histplot(df['H_normalized'], bins=nbins, ax=axs[1][3])
fig.tight_layout()
fig.savefig(SAVE_DIR / 'normalized_lcms_histograms.png', dpi=300)

In [ ]:
nbins = 50
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(7.5, 3.75))
sns.histplot(df['A_normalized'] + 1e-5, bins=nbins, ax=axs[0][0], log_scale=(True, True))
sns.histplot(df['B_normalized'] + 1e-5, bins=nbins, ax=axs[0][1], log_scale=(True, True))
sns.histplot(df['C_normalized'] + 1e-5, bins=nbins, ax=axs[0][2], log_scale=(True, True))
sns.histplot(df['D_normalized'] + 1e-5, bins=nbins, ax=axs[0][3], log_scale=(True, True))
sns.histplot(df['E_normalized'] + 1e-5, bins=nbins, ax=axs[1][0], log_scale=(True, True))
sns.histplot(df['F_normalized'] + 1e-5, bins=nbins, ax=axs[1][1], log_scale=(True, True))
sns.histplot(df['G_normalized'] + 1e-5, bins=nbins, ax=axs[1][2], log_scale=(True, True))
sns.histplot(df['H_normalized'] + 1e-5, bins=nbins, ax=axs[1][3], log_scale=(True, True))
for ax in axs.flatten():
    ax.set_xlim(1e-5, 1)
    ax.set_ylim(1, 5e4)
fig.tight_layout()
fig.savefig(SAVE_DIR / 'normalized_lcms_histograms_logscale.png', dpi=300)

In [ ]:
# let's see product A LC-MS response histogram with log axes
plt.figure()
sns.histplot(df['A_normalized'] + 1e-5, bins=50, log_scale=(True, True))  # +1e-5 to avoid log(0)=-inf
plt.title('Log-log histogramm of product A LC-MS ratios')
plt.savefig(SAVE_DIR / 'normalized_lcms_product-A_histogram_logscale.png', dpi=300)

In [ ]:
# let's see product A LC-MS response in a cumulative histogram
plt.figure()
sns.displot(df['A_normalized'], cumulative=True)
plt.title('Cumulative histogramm of product A LC-MS ratios')
plt.savefig(SAVE_DIR / 'normalized_lcms_product-A_cumulative_histogram.png', dpi=300)

In [ ]:
# let's enlarge the 0 to 1e-2 range
plt.figure()
sns.displot(df.loc[df["A_normalized"] < 0.01, 'A_normalized'], cumulative=True, bins=1000)
plt.title('Cumulative histogram of product A LC-MS ratios, cutout 0 to 0.01')
plt.savefig(SAVE_DIR / 'normalized_lcms_product-A_cumulative_histogram_cutout_0to1e-2.png', dpi=300)

In [ ]:
# let's enlarge the 0 to 1e-3 range
plt.figure()
sns.displot(df.loc[df["A_normalized"] < 0.001, 'A_normalized'], cumulative=True, bins=1000)
plt.title('Cumulative histogram of product A LC-MS ratios')
plt.axvline(x=0.0002, color=bode_palette.orange)
plt.show()

### Conclusion from LCMS threshold search
The LC-MS threshold can be set to 2e-4 for product A, as this is the point where the curve starts to flatten out. However, there are very few samples in this region, and it may not make sense to exclude them, particularly given that we already have noise filtering in the LC-MS peak picking step. It would make more sense to include everything >0 as positive, but use the lcms ratios for the other products to determine when only a meaningless amount of A is formed.

In [ ]:
# Curious side question: What products have the highest lcms response for product A?

products = []
for vl_id in df.sort_values('A_normalized', ascending=False).head(5)['vl_id']:
    products.append(db.get_vl_member(vl_id))
[Compute2DCoords(p) for p in products]
Draw.MolsToGridImage(products)

### Building Block statistics <a id=4></a>

Let's look into details: __For each initator, how many reactions have worked above the threshold?__

In [ ]:
THRESHOLD = 0  # response has to be bigger than this to be counted as success

In [ ]:
# first add a success True/False column
for product in "ABCDEFGH":
    df[f"{product}_success"] = df[f"{product}_normalized"] > THRESHOLD

In [ ]:
def calc_success_rate(df, column, product_type="A", sort_results=True):
    """
    Take a dataframe df, where for one column of interest, for each unique identifier in that column, the frequency
    of successful reactions (as indicated by the threshold) is calculated.
    :param df: pandas.DataFrame
    :param column: Column by which to group, e.g. monomer or monomer_type
    :param product_type: Which product to use for success rate calculation
    :return: pandas.DataFrame
    """
    # group by the column of interest
    grouped = df.groupby([column])
    # count positive + all examples and determine success rate
    n_all = grouped[f"{product_type}_success"].count().rename('n_all')
    n_success = grouped[f"{product_type}_success"].sum().rename('n_success')
    success_rates = (n_success / n_all).rename('success_rate')
    # assemble results
    df = pd.concat([n_all, n_success, success_rates], axis=1)
    df.index.name = "building_block"
    if sort_results is True:
        df = df.sort_index(
            #key=alphanumeric_index_sort
        )
    df = df.reset_index()
    return df

In [ ]:
# obtain dictionary of building block classes from database
reactant_class = {f"I{i}": db.get_building_block_class(f"I{i}") for i in range(1, 79)}
reactant_class.update({f"M{i}": db.get_building_block_class(f"M{i}") for i in range(1, 75)})
reactant_class.update({f"T{i}": db.get_building_block_class(f"T{i}") for i in range(1, 42)})

# assign colors to the different classes for plotting
reactant_colors = {
    "KAT_hetarom": "blue",
    "KAT_arom": "lightblue",
    "KAT_al": "green",
    "Mon_fused": "blue",
    "Mon_spiro_2": "green",
    "Mon_spiro_3": "lightgreen",
    "Mon_sub_2": "red",
    "Mon_sub_3": "orange",
    "TerABT": "blue",
    "TerTH": "green",
}

building_block_colors = {k: reactant_colors[v] for k, v in reactant_class.items()}
#building_block_colors

In [ ]:
# we calculate success rates for product A for all initiators
success_rates_by_initiator = calc_success_rate(df, 'initiator_long')
# let's do the same for monomers
success_rates_by_monomer = calc_success_rate(df, 'monomer_long')
# and the same for terminators
success_rates_by_terminator = calc_success_rate(df, 'terminator_long')

success_rates = pd.concat((success_rates_by_initiator, success_rates_by_monomer, success_rates_by_terminator))
print("Success rates:")
for _, i in success_rates.iterrows():
    print(f'{i["building_block"]}: {i["success_rate"]:.1%} ({i["n_success"]}/{i["n_all"]})')


In [ ]:
def plot_success_count_by_building_block(success_rates, figsize=(4, 6), save_path=None):
    # show how many reactions are successful for each initiator
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=figsize, width_ratios=[3, 1])

    # plot a mean line
    axs[0].axvline(success_rates['success_rate'].mean(), ls='--', c='black')
    # plot the success rates
    sns.barplot(x=success_rates['success_rate'], y=success_rates['building_block'], ax=axs[0])
    # plot the number of experiments
    sns.barplot(x=success_rates['n_all'], y=success_rates['building_block'], ax=axs[1])

    # set details
    axs[0].set_xlabel(f'Ratio of successful reactions')
    axs[0].set_ylabel('Building block')
    axs[0].set_xlim(0,1)
    axs[1].set_xlabel('Number of reactions')
    axs[1].set_ylabel(None)
    axs[1].set_yticks([])
    fig.tight_layout()
    if save_path is not None:
        fig.savefig(save_path, dpi=300)
    return fig

In [ ]:
# save to file but don't show
plot_success_count_by_building_block(success_rates_by_initiator, save_path=SAVE_DIR / 'success_rates_by_initiator.png')
plot_success_count_by_building_block(success_rates_by_monomer, save_path=SAVE_DIR / 'success_rates_by_monomer.png')
plot_success_count_by_building_block(success_rates_by_terminator, save_path=SAVE_DIR / 'success_rates_by_terminator.png', figsize=(4, 4))

In [ ]:
def plot_outcome_2D(df, x1, x2, y, product, building_block_colors=None, save_dir=SAVE_DIR):
    """Plot a 2D heatmap of the success rate of reactions for two building block types averaged over the third type"""
    # set up layout
    rel_width = (df[x1].nunique(), df[x2].nunique())
    rel_height = df[y].nunique() / (df[x1].nunique() + df[x2].nunique())
    fig, ax = plt.subplots(figsize=(18, 18* rel_height), ncols=2, width_ratios=rel_width)
    cbar_ax = fig.add_axes([0.92, 0.3, 0.015, 0.4])  # [left, bottom, width, height]

    # prepare data
    df_heatmap1 = df.groupby([y, x1]).mean(numeric_only=True)[f"{product}_success"].unstack()
    df_heatmap1 = df_heatmap1.sort_index(key=alphanumeric_index_sort)
    df_heatmap1 = df_heatmap1.sort_index(key=alphanumeric_index_sort, axis=1)

    df_heatmap2 = df.groupby([y, x2]).mean(numeric_only=True)[f"{product}_success"].unstack()
    df_heatmap2 = df_heatmap2.sort_index(key=alphanumeric_index_sort)
    df_heatmap2 = df_heatmap2.sort_index(key=alphanumeric_index_sort, axis=1)

    # plot
    sns.heatmap(df_heatmap1, ax=ax[0],  cmap='viridis', vmin=0, vmax=1, center=0, linewidths=0.5, annot=False, fmt='.0%', cbar_ax=cbar_ax)
    sns.heatmap(df_heatmap2, ax=ax[1],  cmap='viridis', vmin=0, vmax=1, center=0, linewidths=0.5, annot=False, fmt='.0%', cbar=False, yticklabels=False)

    ax[0].set_yticklabels(ax[0].get_yticklabels(), rotation=0)
    if building_block_colors:
        for axes in ax:
            # Set the x-axis tick label background colors for the first heatmap
            for label in axes.get_xticklabels():
                bbox = dict(boxstyle="round", fc=building_block_colors[label.get_text()], alpha=0.5)
                label.set_bbox(bbox)
            # Set the y-axis tick label background colors for the first heatmap
            for label in axes.get_yticklabels():
                bbox = dict(boxstyle="round", fc=building_block_colors[label.get_text()], alpha=0.5)
                label.set_bbox(bbox)

    ax[0].set_xlabel(x1.capitalize())
    ax[0].set_ylabel(y.capitalize())
    ax[1].set_xlabel(x2.capitalize())
    ax[1].set_ylabel(None)
    if save_dir:
        fig.savefig(save_dir / f"heatmap_{y}_product{product}.png", dpi=300, bbox_inches="tight")
    return fig


In [ ]:
%%capture
# save to file but don't show
for product in "ABCDEFGH":
    plot_outcome_2D(df, "monomer", "terminator", "initiator", product, building_block_colors=building_block_colors)
    plot_outcome_2D(df, "initiator", "terminator", "monomer", product, building_block_colors=building_block_colors)
    plot_outcome_2D(df, "initiator", "monomer", "terminator", product, building_block_colors=building_block_colors)

### Trends within building block classes <a id=5></a>
Let's look at how well different types of initiators, monomers, terminators react

In [ ]:
# let's check Initiators
success_rates_by_initiator_type = calc_success_rate(df, 'initiator_type', sort_results=False)
for _, i in success_rates_by_initiator_type.iterrows():
    print(f'Success rate for {i["building_block"]}: {i["success_rate"]:.1%} ({i["n_success"]}/{i["n_all"]})')

In [ ]:
# let's check Monomers
success_rates_by_monomer_type = calc_success_rate(df, 'monomer_type', sort_results=False)
for _, i in success_rates_by_monomer_type.iterrows():
    print(f'Success rate for {i["building_block"]}: {i["success_rate"]:.1%} ({i["n_success"]}/{i["n_all"]})')

In [ ]:
# let's check Terminators
success_rates_by_terminator_type = calc_success_rate(df, 'terminator_type',  sort_results=False)
for _, i in success_rates_by_terminator_type.iterrows():
    print(f'Success rate for {i["building_block"]}: {i["success_rate"]:.1%} ({i["n_success"]}/{i["n_all"]})')

## Scaling the LCMS responses <a id=6></a>
- It seems to make sense to use some kind of robust scaling as we definitely expect outliers in LCMS data.
- We do not require centering, as the data starts at 0 and is not normally distributed.
- We could divide by the interquartile range (IQR) to get a robust scaling.
- We could also use the median absolute deviation (MAD) to get a robust scaling.
- However, both above ideas seem more suitable for symmetric distributions, while we have a highly skewed distribution with a defined minimum at 0.
- Therefore, we could just divide by, say, the 90th percentile to get the equivalent of min-max scaling but robust to outliers.
- Which percentile we choose, depends on the number of outliers we expect. The 90th percentile seems reasonable as we already remove internal standard errors and we can expect less than 10% of compounds to be outliers by virtue of ionizability alone. At the same time, choosing the 90th means we expect all products to form in at least 10% of the reactions.
- To not be dependent on how often a product forms: Use a lower (say 80th) percentile, but use only non-zero values in calculating the percentile.

### An extension to the scaling idea
Assume we have achieved robust scaling. There remains a problem:
- Consider product A systematically ionizes 10 times better than product B.
- Further, consider a reaction where we measure a non-zero response for product A that is at the lower detection limit
- It follows that we have no way to tell whether product B was formed or not, we only know it was not formed 10 times as much as A.

In general, if product A ionized better than product B by a factor of x, we need to see a response for product A exceeding the lower detection limit by a factor of x to conclude that product A is the major product.


In [ ]:
def robust_scale(x: np.ndarray, percentile: int = 75, return_scaling_factor=False) -> Union[np.ndarray, Tuple[np.ndarray, float]]:
    """
    Robust scaling of a numpy array by dividing by the value for a percentile.
    The percentile is calculated from all non-zero values.
    """
    x = x.copy()
    scaling_factor = np.percentile(x[x > 0], percentile)
    x /= scaling_factor
    if return_scaling_factor:
        return x, scaling_factor
    else:
        return x

In [ ]:
# before we apply scaling, we need to verify how sensitive the scaling is to our choice of the percentile.
factors = []
for percentile in range(1, 100):
    scaled_arrs = []
    scaling_factors = []
    for s in "ABCDEFGH":
        arr, factor = robust_scale(df[f'product_{s}_lcms_ratio'].values, percentile=percentile, return_scaling_factor=True)
        scaled_arrs.append(arr)
        scaling_factors.append(factor)
    factors.append(scaling_factors / scaling_factors[0])

In [ ]:
# plot the relative scaling factors for each product (A is always 1) for all percentiles between [1, 99]
plt.figure(figsize=(7.5,5))
plt.plot(factors)
plt.xlabel("Percentile")
plt.ylabel("Scaling factor, relative to A")
plt.axvline(85, color="red", linestyle="--", label="85th")
plt.legend("ABCDEFGH")
plt.savefig(SAVE_DIR / "scaling_factors_by_percentile.png", dpi=300)

### Result of the percentile/scaling factor analysis
This outcome is good. The range of the scaling factors is mostly 2x - 3x. This seems reasonable given that the molecules always share quite a few residues. Also, except product D, all relative factors are quite stable w.r.t. the choice of percentile. In particular, relative A,B,C are stable. As we expect, in the very high percentiles (>95 for most, >90 for E), the scaling factor becomes unstable as it is now dominated by outliers.

**Based on the plot, we choose the 85th percentile**

In [ ]:
# apply scaling to all products and show scaling factors for control
scaling_factors = []
for s in "ABCDEFGH":
        arr, factor = robust_scale(df[f'product_{s}_lcms_ratio'].values, percentile=85, return_scaling_factor=True)
        df[f'{s}_scaled'] = arr
        scaling_factors.append(factor)
print("Scaling factors relative to A:")
for s, i in zip("ABCDEFGH", scaling_factors):
    print(f"{s}: {i/scaling_factors[0]:.1f}")

### Discussion of scaling factors
From the values of the scaling factors, products A, B, E, and  F ionize on a similar scale while product D ionizes 5x better and products C, G, and H ionize 5x worse.

Does this make sense bases on chemical intuition?
It's hard to predict ionization in MS, but the good ionization of D seems plausible as it is rather small and has a lot of polar groups. H is surprising because a similar argument could be made, but then again, H forms under very specific circumstances, so the side chain diversity is limited and that may bias the ionizability.

In [ ]:
df[["A_scaled", "B_scaled", "C_scaled", "D_scaled", "E_scaled", "F_scaled", "G_scaled", "H_scaled"]].describe()

## Outcome for the main product <a id=7></a>

In [ ]:
# assign the main product
df["main_product"] = df[["A_scaled", "B_scaled", "C_scaled"]].idxmax(axis=1).str.replace("_scaled", "")
# are there any reactions where neither A,nor B, nor C appear?
df.loc[df[["A_scaled", "B_scaled", "C_scaled"]].sum(axis=1) == 0, "main_product"] = "none"
df

In [ ]:
# how are the different success rates related?
# scatterplot between A_scaled and others
x = df["A_scaled"].values

fig, ax = plt.subplots(figsize=(10,10), ncols=3, nrows=3)
for i, s in enumerate("BCDEFGH"):
    row = i // 3
    col = i % 3
    y = np.nan_to_num(df[f"{s}_scaled"].values)
    # Calculate the point density
    xy = np.vstack([x,y])
    z = np.log10(gaussian_kde(xy)(xy))

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    ax[row, col].scatter(x, y, c=z, s=5, cmap="viridis")
    ax[row, col].set_xlabel('A_scaled')
    ax[row, col].set_ylabel(f'{s}_scaled')
    ax[row, col].set_xlim(0, 10)  # exclude outliers. We scaled the data to be able to compared them
    ax[row, col].set_ylim(0, 10)  # exclude outliers. We scaled the data to be able to compared them
# Hide the last two axes
ax[2, 1].set_visible(False)
ax[2, 2].set_visible(False)
plt.tight_layout()
plt.savefig(SAVE_DIR / 'LCMS_yields_scatter.png', dpi=300, bbox_inches='tight')

In [ ]:
# among A,B,C, which is the major product?
def plot_main_product_2D(df, x1, x2, y, building_block_colors=None, save_dir=SAVE_DIR):
    """Plot a 2D heatmap of the success rate of reactions for two building block types averaged over the third type"""
    rel_width = (df[x1].nunique(), df[x2].nunique())
    rel_height = df[y].nunique() / (df[x1].nunique() + df[x2].nunique())
    fig, ax = plt.subplots(figsize=(14, 14 * rel_height), ncols=2, width_ratios=rel_width)
    cbar_ax = fig.add_axes([0.92, 0.1, 0.015, 0.8])  # [left, bottom, width, height]
    df_heatmap1 = df.groupby([y, x1])["main_product"].describe()["top"].map({"A": 0, "B": 1, "C": 2, "none": 3}).unstack()
    df_heatmap1 = df_heatmap1.sort_index(key=alphanumeric_index_sort)
    df_heatmap1 = df_heatmap1.sort_index(key=alphanumeric_index_sort, axis=1)

    df_heatmap2 = df.groupby([y, x2])["main_product"].describe()["top"].map({"A": 0, "B": 1, "C": 2, "none": 3}).unstack()
    df_heatmap2 = df_heatmap2.sort_index(key=alphanumeric_index_sort)
    df_heatmap2 = df_heatmap2.sort_index(key=alphanumeric_index_sort, axis=1)

    # Plot the first heatmap
    sns.heatmap(df_heatmap1, ax=ax[0], cmap=sns.color_palette("colorblind", n_colors=4), linewidths=0.5, annot=False, fmt='.0%', cbar_ax=cbar_ax, xticklabels=True)

    # Plot the second heatmap
    sns.heatmap(df_heatmap2, ax=ax[1], cmap=sns.color_palette("colorblind", n_colors=4), linewidths=0.5, annot=False, fmt='.0%', cbar=False, xticklabels=True, yticklabels=False)
    
    ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
    ax[0].set_yticklabels(ax[0].get_yticklabels(), rotation=0)
    ax[0].set_xlabel(ax[0].get_xlabel().capitalize())
    ax[0].set_ylabel(ax[0].get_ylabel().capitalize())
    ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
    ax[1].set_xlabel(ax[1].get_xlabel().capitalize())
    ax[1].set_ylabel(None)

    # Set the tick label background colors
    if building_block_colors:
        for axes in ax:
            for label in axes.get_xticklabels():
                bbox = dict(boxstyle="Round,pad=0.1", fc=building_block_colors[label.get_text()], alpha=0.5)
                label.set_bbox(bbox)
            # Set the y-axis tick label background colors for the first heatmap
            for label in axes.get_yticklabels():
                bbox = dict(boxstyle="Round,pad=0.1", fc=building_block_colors[label.get_text()], alpha=0.5)
                label.set_bbox(bbox)

    # change the colorbar labels to convey their meaning
    cbar_ax.set_ylabel("Main product")
    cbar_ax.yaxis.set_label_position("left")
    cbar_ax.set_yticks([i * 3 * 0.25 for i in (0.5, 1.5, 2.5, 3.5)], ["A", "B", "C", "none"], rotation=0)
    #cbar_ax.set_yticks([])  # for slides, remove the ticks
    cbar_ax.set_aspect(10)

    if save_dir:
        fig.savefig(save_dir / f"heatmap_{y}_main_product.png", dpi=300, bbox_inches="tight")
    return fig

In [ ]:
%%capture
plot_main_product_2D(df, "monomer", "terminator", "initiator", building_block_colors=building_block_colors)
plot_main_product_2D(df, "initiator", "terminator", "monomer", building_block_colors=building_block_colors)
plot_main_product_2D(df, "initiator", "monomer", "terminator", building_block_colors=building_block_colors)

### Plot with relative success rates
The plots above exclude some information because they only show the most successful product. We can also plot the relative success rates of the different products.

In [ ]:
grouped = df.groupby(["initiator", "monomer"])
all_count = grouped["main_product"].count()
a_ratio = grouped["main_product"].sum().str.count("A") / all_count
b_ratio = grouped["main_product"].sum().str.count("B") / all_count
c_ratio = grouped["main_product"].sum().str.count("C") / all_count
none_ratio = grouped["main_product"].sum().str.count("none") / all_count
relative_products = pd.concat((a_ratio, b_ratio, c_ratio, 1 - none_ratio), axis=1, ignore_index=True)  # we do 1 - none_ratio because we will use that for alpha
relative_products = relative_products.unstack()
relative_products = relative_products.sort_index(key=alphanumeric_index_sort)
relative_products = relative_products.sort_index(key=alphanumeric_index_sort, axis=1, level=1, sort_remaining=False)
arr = relative_products.to_numpy()
heatmap_arr = arr.reshape((len(relative_products.index.get_level_values(0).unique()), len(relative_products.columns.get_level_values(1).unique()), 4), order="C")
heatmap_arr

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), ncols=2, width_ratios=(.8, .2))
ax[1].axis("off")
ax = ax[0]
legend_ax = fig.add_axes([0.8, 0.4, 0.18, 0.2])  # [left, bottom, width, height]
im = ax.imshow(heatmap_arr, interpolation='nearest')
ax.set_xticks(np.arange(heatmap_arr.shape[1]), labels=relative_products.columns.get_level_values(1).unique(), rotation=90)
ax.set_yticks(np.arange(heatmap_arr.shape[0]), labels=relative_products.index)

legend_ax.add_patch(Rectangle((0, 0), .1, .1, color=(1,0,0,1)))
legend_ax.text(0.05, 0.05, "A", fontsize=12, ha="center", va="center")
legend_ax.add_patch(Rectangle((0, .1), .1, .1, color=(0,1,0,1)))
legend_ax.text(0.05, 0.15, "B", fontsize=12, ha="center", va="center")
legend_ax.add_patch(Rectangle((0, .2), .1, .1, color=(0,0,1,1)))
legend_ax.text(0.05, 0.25, "C", fontsize=12, ha="center", va="center")

legend_ax.add_patch(Rectangle((.1, 0), .1, .1, color=(.5,.5,0,1)))
legend_ax.text(0.15, 0.05, "A+B", fontsize=12, ha="center", va="center")
legend_ax.add_patch(Rectangle((.1, .1), .1, .1, color=(.5,0,.5,1)))
legend_ax.text(0.15, 0.15, "A+C", fontsize=12, ha="center", va="center")
legend_ax.add_patch(Rectangle((.1, .2), .1, .1, color=(0,.5,.5,1)))
legend_ax.text(0.15, 0.25, "B+C", fontsize=12, ha="center", va="center")

legend_ax.add_patch(Rectangle((.2, .0), .1, .1, color=(.33,.33,.33,1)))
legend_ax.text(0.25, 0.05, "A+B+C", fontsize=12, ha="center", va="center")
legend_ax.set_xlim(0, 0.3)
legend_ax.set_ylim(0, 0.3)
legend_ax.axis("off")
legend_ax.set_title("Legend", fontsize=12)
fig.tight_layout()
fig.savefig(SAVE_DIR / "heatmap_IM_relative_success.png", dpi=300, bbox_inches="tight")

In [ ]:
# Can we plot the same information better by using tiny pie charts as the heatmap?
def heatmap_with_pies(ax, data_array, row_labels, column_labels, cmap=None, norm=None, building_block_colors=None):

    pies=[]
    # generate all the individual pies
    for row_index, row in enumerate(row_labels):
        for column_index, column in enumerate(column_labels):
            data_slice = data_array[row_index, column_index]
            radius = .4 * data_slice[:3].sum() +.05
            if np.isnan(data_slice).any():
                # if there are nan value, we have not conducted this reaction, so we plot nothing
                continue
            if data_slice.sum() == 0:
                # if there are no products found, we plot a black circle
                wedges, _ = plt.pie((1,), center=(column_index, row_index), radius=radius, colors=("black",), wedgeprops={"linewidth": 0})
            else:
                wedges, _ = plt.pie(data_slice, center=(column_index, row_index), radius=radius, colors=sns.color_palette("colorblind", n_colors=4), wedgeprops={"linewidth": 0}, startangle=270)

    col = PatchCollection(pies, array=data_array.flatten(), cmap=cmap, norm=norm)

    ax.add_collection(col)

    ax.set_xticks(np.arange(data_array.shape[1]), labels=column_labels, rotation=90)
    ax.set_yticks(np.arange(data_array.shape[0]), labels=row_labels)

    # we invert the y-axis (so that the first building block will be at the top)
    ax.invert_yaxis()

    # put a legend, based only on the last wedges (faster than automatic detection)
    ax.legend(wedges, ("A", "B", "C", "none"), loc="upper left", bbox_to_anchor=(1, 1), title="Products")

    # Set the tick label background colors
    if building_block_colors:
        for label in ax.get_xticklabels():
            bbox = dict(boxstyle="round", fc=building_block_colors[label.get_text()], alpha=0.5)
            label.set_bbox(bbox)
        # Set the y-axis tick label background colors for the first heatmap
        for label in ax.get_yticklabels():
            bbox = dict(boxstyle="round", fc=building_block_colors[label.get_text()], alpha=0.5)
            label.set_bbox(bbox)

    return ax

In [ ]:
%%capture
# plot for initiator and monomer
data_array=heatmap_arr.copy()
data_array[:,:,3] = 1 - data_array[:,:,3]  # we need the success rate not the failure rate
row_labels=relative_products.index
column_labels=relative_products.columns.get_level_values(1).unique()

fig, ax = plt.subplots(figsize=(11, 10))
heatmap_with_pies(ax, data_array, row_labels, column_labels, building_block_colors=building_block_colors)
ax.set_xlabel("Monomer")
ax.set_ylabel("Initiator")

plt.tight_layout()
plt.savefig(SAVE_DIR / "heatmap_IM_relative_success_pies.png", dpi=300, bbox_inches="tight")

In [ ]:
# prepare data for initiator and terminator
grouped = df.groupby(["initiator", "terminator"])
all_count = grouped["main_product"].count()
a_ratio = grouped["main_product"].sum().str.count("A") / all_count
b_ratio = grouped["main_product"].sum().str.count("B") / all_count
c_ratio = grouped["main_product"].sum().str.count("C") / all_count
none_ratio = grouped["main_product"].sum().str.count("none") / all_count
relative_products = pd.concat((a_ratio, b_ratio, c_ratio, 1 - none_ratio), axis=1, ignore_index=True)  # we do 1 - none_ratio because we will use that for alpha
relative_products = relative_products.unstack()
relative_products = relative_products.sort_index(key=alphanumeric_index_sort)
relative_products = relative_products.sort_index(key=alphanumeric_index_sort, axis=1, level=1, sort_remaining=False)
arr = relative_products.to_numpy()
heatmap_arr = arr.reshape((len(relative_products.index.get_level_values(0).unique()), len(relative_products.columns.get_level_values(1).unique()), 4), order="C")
heatmap_arr

In [ ]:
%%capture
# plot for initiator and terminator
data_array=heatmap_arr.copy()
data_array[:,:,3] = 1 - data_array[:,:,3]  # we need the success rate not the failure rate
row_labels=relative_products.index
column_labels=relative_products.columns.get_level_values(1).unique()

fig, ax = plt.subplots(figsize=(11, 10))
heatmap_with_pies(ax, data_array, row_labels, column_labels, building_block_colors=building_block_colors)
ax.set_xlabel("Terminator")
ax.set_ylabel("Initiator")

plt.tight_layout()
plt.savefig(SAVE_DIR / "heatmap_IT_relative_success_pies.png", dpi=300, bbox_inches="tight")

In [ ]:
# prepare data for monomer and terminator
grouped = df.groupby(["monomer", "terminator"])
all_count = grouped["main_product"].count()
a_ratio = grouped["main_product"].sum().str.count("A") / all_count
b_ratio = grouped["main_product"].sum().str.count("B") / all_count
c_ratio = grouped["main_product"].sum().str.count("C") / all_count
none_ratio = grouped["main_product"].sum().str.count("none") / all_count
relative_products = pd.concat((a_ratio, b_ratio, c_ratio, 1 - none_ratio), axis=1, ignore_index=True)  # we do 1 - none_ratio because we will use that for alpha
relative_products = relative_products.unstack()
relative_products = relative_products.sort_index(key=alphanumeric_index_sort)
relative_products = relative_products.sort_index(key=alphanumeric_index_sort, axis=1, level=1, sort_remaining=False)
arr = relative_products.to_numpy()
heatmap_arr = arr.reshape((len(relative_products.index.get_level_values(0).unique()), len(relative_products.columns.get_level_values(1).unique()), 4), order="C")
heatmap_arr

In [ ]:
%%capture
# plot for monomer and terminator
data_array=heatmap_arr.copy()
data_array[:,:,3] = 1 - data_array[:,:,3]  # we need the success rate not the failure rate
row_labels=relative_products.index
column_labels=relative_products.columns.get_level_values(1).unique()

fig, ax = plt.subplots(figsize=(11, 10))
heatmap_with_pies(ax, data_array, row_labels, column_labels, building_block_colors=building_block_colors)
ax.set_xlabel("Terminator")
ax.set_ylabel("Monomer")

plt.tight_layout()
plt.savefig(SAVE_DIR / "heatmap_MT_relative_success_pies.png", dpi=300, bbox_inches="tight")

## Outcomes for repeated reactions
We ended up creating some duplicate reactions (divergence from the original duplicate-free plan became necessary when we ran out of some building blocks). Let's check how well they reproduce.

In [ ]:
duplicated = df.loc[df.duplicated(subset=["long_name"], keep=False)]

In [ ]:
# number of duplicated reactions
grouped_duplicates = duplicated.groupby("long_name")
len(grouped_duplicates)

In [ ]:
grouped_duplicates_unstacked = grouped_duplicates["A_scaled"].apply(lambda x: pd.Series(x.values)).unstack()
grouped_duplicates_unstacked

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 7.5), nrows=3, ncols=3)
fig.subplots_adjust(left=0.15)
product_data = []  # we will want to re-use the data later, so we catch it here
for i, s in enumerate("ABCDEFGH"):
    row = i // 3
    col = i % 3
    
    # extract data
    data = grouped_duplicates[f"{s}_scaled"].apply(lambda x: pd.Series(x.values)).unstack()
    product_data.append(data)  # not used for plotting, just later re-use
    
    # plot data
    ax[row, col] = sns.scatterplot(data=data, x=0, y=1, ax=ax[row, col], s=8, color=bode_palette.blue)
    # plot diagonal (in data coords)
    ax[row, col].plot(ax[row, col].get_ylim(), ax[row,col].get_ylim(), c=bode_palette.orange, ls=":")
    # plot R2 score
    ax[row, col].text(x=(ax[row, col].get_xlim()[1] * .8), y=.1, s=f"$R^2$={r2_score(data[0], data[1]):.2f}")
    # plot a label for the axes
    ax[row, col].text(x=-0.1, y=1.1, s=s, fontsize=12, fontweight="bold", ha="center", va="center", transform=ax[row,col].transAxes)
    
    # set axis labels
    if col == 0:
        ax[row, col].set_ylabel("Replicate 2")
    else:
        ax[row, col].set_ylabel(None)
    if row == 2:
        ax[row, col].set_xlabel("Replicate 1")
    else:
        ax[row, col].set_xlabel(None)
    
ax[2, 2].set_visible(False)
ax[1, 2].set_xlabel("Replicate 1")
fig.tight_layout()
fig.savefig(SAVE_DIR / "replicates_scaled_responses_scatter.png", dpi=300)

In [ ]:
# at a specific threshold for binning into positive and negative class, how often do the labels have the same class?
accuracies = []
for s, data in zip("ABCDEFGH", product_data):
    accuracy = accuracy_score((data > 0)[0], (data > 0)[1])
    accuracies.append(accuracy)
    print(f"Product {s}: {accuracy:.2f}")

In [ ]:
# plot this in a bar plot
fig, ax = plt.subplots(figsize=(3.6, 2.4))
sns.barplot(x=list("ABCDEFGH"), y=accuracies, ax=ax, color=bode_palette.blues[2])
ax.axhline(1, ls="--", c="black")
ax.set_xlabel("Product")
ax.set_ylabel("Accuracy")
ax.set_ylim(0,1)
fig.tight_layout()
fig.savefig(SAVE_DIR / "replicates_binned_accuracies.png", dpi=300)
fig.savefig(SAVE_DIR / "replicates_binned_accuracies.svg", transparent=True)

In [ ]:
grouped_main_products = grouped_duplicates["main_product"].apply(lambda x: pd.Series(x.values)).unstack()

In [ ]:
conf_mat = confusion_matrix(grouped_main_products[0], grouped_main_products[1])
conf_mat

In [ ]:
fig, ax = plt.subplots(figsize=(2.4, 2.4))
axislabels = ["A", "B", "C", "None"]
ax = sns.heatmap(conf_mat, cmap="viridis", annot=True, fmt="d", xticklabels=axislabels, yticklabels=axislabels, ax=ax, cbar=False)
ax.set_xlabel("Replicate 1")
ax.set_ylabel("Replicate 2")
ax.set_yticks(ax.get_yticks(), ax.get_yticklabels(), rotation=0)
ax.set_aspect(1)
fig.tight_layout()
fig.savefig(SAVE_DIR / "replicates_confusion_matrix.png", dpi=300)
fig.savefig(SAVE_DIR / "replicates_confusion_matrix.svg", transparent=True)


In [ ]:
balanced_accuracy_score(grouped_main_products[0], grouped_main_products[1])  # nb, this is average (macro) recall

In [ ]:
recall_score(grouped_main_products[0], grouped_main_products[1], average="weighted")